<a href="https://colab.research.google.com/github/clustering-jun/GNU-MachineLearning/blob/main/L11-3_%EB%AA%A8%EB%8D%B8_%ED%8F%89%EA%B0%80_%EB%B0%8F_%EC%84%A0%ED%83%9D%EC%9D%98_%ED%99%9C%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **분류 문제 사례: 신용 카드 사기 감지 데이터**

In [2]:
import pandas as pd

df = pd.read_csv('creditcard.csv')
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0


In [3]:
df.isnull().sum().max()

1

In [4]:
df.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

In [5]:
from sklearn.preprocessing import RobustScaler

rob_scaler = RobustScaler()

df['scaled_amount'] = rob_scaler.fit_transform(df['Amount'].values.reshape(-1, 1))
df['scaled_time'] = rob_scaler.fit_transform(df['Time'].values.reshape(-1, 1))

df.drop(['Time', 'Amount'], axis = 1, inplace = True)

In [6]:
scaled_amount = df['scaled_amount']
scaled_time = df['scaled_time']

df.drop(['scaled_amount', 'scaled_time'], axis=1, inplace=True)
df.insert(0, 'scaled_amount', scaled_amount)
df.insert(1, 'scaled_time', scaled_time)

df.head()


,scaled_amount,scaled_time,V1,V2,V3,V4,V5,V6,V7,V8,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class
0,1.732950,-1.502748,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0.0
1,-0.277174,-1.502748,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0.0
2,4.866407,-1.502725,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0.0
3,1.375607,-1.502725,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0.0
4,0.643546,-1.502703,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0.0


In [7]:
df = df.sample(frac=1)

fraud_df = df.loc[df['Class'] == 1]
non_fraud_df = df.loc[df['Class'] == 0][:492]

normal_distributed_df = pd.concat([fraud_df, non_fraud_df])

new_df = normal_distributed_df.sample(frac=1, random_state=0)

new_df.head()

,scaled_amount,scaled_time,V1,V2,V3,V4,V5,V6,V7,V8,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class
59539,-0.044052,-0.398281,-2.139051,1.394368,-0.612035,1.049327,-1.162102,-0.768219,-1.997237,0.574997,...,0.025427,0.696955,0.740003,-0.155115,-0.050607,0.268368,-0.469433,-0.405814,-0.152171,1.0
64329,1.053971,-0.347942,-9.848776,7.365546,-12.898538,4.273323,-7.611991,-3.427045,-8.350808,6.863604,...,0.832398,0.931958,-0.874467,-0.192639,-0.035426,0.538665,-0.263934,1.134095,0.225973,1.0
184836,-0.220672,1.353359,-5.972795,4.263953,-3.328564,-3.582345,-3.076257,-1.286651,-2.712963,3.737358,...,-0.297015,0.148563,-0.457296,0.547108,-0.448519,0.008537,-0.834705,-0.942862,0.094398,0.0
6870,-0.300294,-1.304895,-1.863756,3.442644,-4.468260,2.805336,-2.118412,-2.332285,-4.261237,1.701682,...,0.360924,0.667927,-0.516242,-0.012218,0.070614,0.058504,0.304883,0.418012,0.208858,1.0
152019,-0.300294,0.682441,-3.705856,4.107873,-3.803656,1.710314,-3.582466,1.469729,-9.621560,-11.913105,...,3.639603,-5.498772,2.941475,0.916236,-0.255504,-0.183835,-0.584539,-0.315484,-0.097223,1.0


In [8]:
print(new_df['Class'].value_counts()/len(new_df))

Class
0.0    0.562929
1.0    0.437071
Name: count, dtype: float64


In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    new_df.drop('Class', axis=1),
    new_df['Class'],
    test_size = 0.2,
    random_state = 2023
)

X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

models = {"Decision Tree": DecisionTreeClassifier(),
          "Random Forest": RandomForestClassifier(),
          "KNN": KNeighborsClassifier(),
          "Logisitic Regression": LogisticRegression(),
          "SVM": SVC()}

from sklearn.model_selection import cross_val_score

for key, model in models.items():
    model.fit(X_train, y_train)
    training_score = cross_val_score(model, X_train, y_train, cv=5)
    print(model.__class__.__name__, "Training Score:", round(training_score.mean(), 3))

DecisionTreeClassifier Training Score: 0.923
RandomForestClassifier Training Score: 0.958
KNeighborsClassifier Training Score: 0.948
LogisticRegression Training Score: 0.95
SVC Training Score: 0.944


In [11]:
from sklearn.model_selection import GridSearchCV

DT_params = {"criterion": ["gini", "entropy"],
             "max_depth": list(range(2, 4, 1)),
             "min_samples_leaf": list(range(5, 7, 1))}
DT_grid = GridSearchCV(DecisionTreeClassifier(), DT_params)
DT_grid.fit(X_train, y_train)

RF_params = {"criterion": ["gini", "entropy"],
             "n_estimators": [100, 200, 500, 1000],
             "max_depth": list(range(2, 4, 1))}
RF_grid = GridSearchCV(RandomForestClassifier(), RF_params)
RF_grid.fit(X_train, y_train)

KNN_params = {"n_neighbors": list(range(2, 5, 1)),
              "algorithm": ['auto', 'ball_tree', 'kd_tree', 'brute']}
KNN_grid = GridSearchCV(KNeighborsClassifier(), KNN_params)
KNN_grid.fit(X_train, y_train)

LR_params = {"penalty": ['l1', 'l2'],
             "C": [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
LR_grid = GridSearchCV(LogisticRegression(), LR_params)
LR_grid.fit(X_train, y_train)

SVM_params = {"C": [0.5, 0.7, 0.9, 1],
              "kernel": ['rbf', 'poly', 'sigmoid', 'linear']}
SVM_grid = GridSearchCV(SVC(), SVM_params)
SVM_grid.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.5, 0.7, 0.9, 1],
                         'kernel': ['rbf', 'poly', 'sigmoid', 'linear']})

In [12]:
DT_score = cross_val_score(DT_grid.best_estimator_, X_train, y_train, cv=5)
print('Decision Tree Cross Validation Score:', round(DT_score.mean(), 3))

RF_score = cross_val_score(RF_grid.best_estimator_, X_train, y_train, cv=5)
print('Random Forest Cross Validation Score:', round(RF_score.mean(), 3))

KNN_score = cross_val_score(KNN_grid.best_estimator_, X_train, y_train, cv=5)
print('KNN Cross Validation Score:', round(KNN_score.mean(), 3))

LR_score = cross_val_score(LR_grid.best_estimator_, X_train, y_train, cv=5)
print('Logistic Regression Cross Validation Score:', round(LR_score.mean(), 3))

SVM_score = cross_val_score(SVM_grid.best_estimator_, X_train, y_train, cv=5)
print('SVM Cross Validation Score:', round(SVM_score.mean(), 3))

Decision Tree Cross Validation Score: 0.953
Random Forest Cross Validation Score: 0.95
KNN Cross Validation Score: 0.954
Logistic Regression Cross Validation Score: 0.956
SVM Cross Validation Score: 0.946


In [13]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

LR_pred = LR_grid.best_estimator_.predict(X_test)

test_cm = confusion_matrix(y_test, LR_pred)
test_acc = accuracy_score(y_test, LR_pred)
test_prc = precision_score(y_test, LR_pred)
test_rcll = recall_score(y_test, LR_pred)
test_f1 = f1_score(y_test, LR_pred)

print(test_cm)
print()
print("정확도는 {}%".format(round(test_acc*100, 2)))
print("정밀도는 {}%".format(round(test_prc*100, 2)))
print("재현율은 {}%".format(round(test_rcll*100, 2)))
print("F1는 {}%".format(round(test_f1*100, 2)))

[[92  1]
 [14 68]]

정확도는 91.43%
정밀도는 98.55%
재현율은 82.93%
F1는 90.07%


### **연습 문제**
- `bodyPerformance.csv`의 `class`변수에 대해 적절한 과정을 거쳐서 가장 최적이라고 판단되는 분류 모델을 훈련한 뒤 성능평가지표를 출력해보시오.


In [16]:
import pandas as pd

body = pd.read_csv('bodyPerformance.csv')

gender_mapping = {'M':0, 'F':1}
class_mapping = {'A':0, 'B':1, 'C':2, 'D':3}

body['gender'] = body['gender'].replace(gender_mapping)
body['class'] = body['class'].replace(class_mapping)

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(body.drop(columns = 'class'), body['class'], test_size = 0.2, random_state=0)

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(x_train)

x_train_std = scaler.transform(x_train)
x_test_std = scaler.transform(x_test)


/tmp/ipython-input-2070061763.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  body['gender'] = body['gender'].replace(gender_mapping)
/tmp/ipython-input-2070061763.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  body['class'] = body['class'].replace(class_mapping)


In [17]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

models = {"Decision Tree": DecisionTreeClassifier(),
          "Random Forest": RandomForestClassifier(),
          "KNN": KNeighborsClassifier(),
          "Logisitic Regression": LogisticRegression(),
          "SVM": SVC()}

from sklearn.model_selection import cross_val_score

for key, model in models.items():
    model.fit(x_train_std, y_train)
    training_score = cross_val_score(model, x_train_std, y_train, cv=5)
    print(model.__class__.__name__, "Training Score:", round(training_score.mean(), 3))

DecisionTreeClassifier Training Score: 0.638
RandomForestClassifier Training Score: 0.729
KNeighborsClassifier Training Score: 0.555


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

LogisticRegression Training Score: 0.587
SVC Training Score: 0.63


In [18]:
from sklearn.model_selection import GridSearchCV

DT_params = {"criterion": ["gini", "entropy"],
             "max_depth": list(range(2, 4, 1)),
             "min_samples_leaf": list(range(5, 7, 1))}
DT_grid = GridSearchCV(DecisionTreeClassifier(), DT_params)
DT_grid.fit(x_train_std, y_train)

RF_params = {"criterion": ["gini", "entropy"],
             "n_estimators": [100, 200, 500, 1000],
             "max_depth": list(range(2, 4, 1))}
RF_grid = GridSearchCV(RandomForestClassifier(), RF_params)
RF_grid.fit(x_train_std, y_train)

KNN_params = {"n_neighbors": list(range(2, 5, 1)),
              "algorithm": ['auto', 'ball_tree', 'kd_tree', 'brute']}
KNN_grid = GridSearchCV(KNeighborsClassifier(), KNN_params)
KNN_grid.fit(x_train_std, y_train)

LR_params = {"penalty": ['l1', 'l2'],
             "C": [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
LR_grid = GridSearchCV(LogisticRegression(), LR_params)
LR_grid.fit(x_train_std, y_train)

SVM_params = {"C": [0.5, 0.7, 0.9, 1],
              "kernel": ['rbf', 'poly', 'sigmoid', 'linear']}
SVM_grid = GridSearchCV(SVC(), SVM_params)
SVM_grid.fit(x_train_std, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.5, 0.7, 0.9, 1],
                         'kernel': ['rbf', 'poly', 'sigmoid', 'linear']})

In [19]:
DT_score = cross_val_score(DT_grid.best_estimator_, x_train_std, y_train, cv=5)
print('Decision Tree Cross Validation Score:', round(DT_score.mean(), 3))

RF_score = cross_val_score(RF_grid.best_estimator_, x_train_std, y_train, cv=5)
print('Random Forest Cross Validation Score:', round(RF_score.mean(), 3))

KNN_score = cross_val_score(KNN_grid.best_estimator_, x_train_std, y_train, cv=5)
print('KNN Cross Validation Score:', round(KNN_score.mean(), 3))

LR_score = cross_val_score(LR_grid.best_estimator_, x_train_std, y_train, cv=5)
print('Logistic Regression Cross Validation Score:', round(LR_score.mean(), 3))

SVM_score = cross_val_score(SVM_grid.best_estimator_, x_train_std, y_train, cv=5)
print('SVM Cross Validation Score:', round(SVM_score.mean(), 3))

Decision Tree Cross Validation Score: 0.556
Random Forest Cross Validation Score: 0.584
KNN Cross Validation Score: 0.54


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Logistic Regression Cross Validation Score: 0.608
SVM Cross Validation Score: 0.661


In [21]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

LR_pred = LR_grid.best_estimator_.predict(x_test_std)

test_cm = confusion_matrix(y_test, LR_pred)
test_acc = accuracy_score(y_test, LR_pred)
test_prc = precision_score(y_test, LR_pred, average='macro')
test_rcll = recall_score(y_test, LR_pred, average='macro')
test_f1 = f1_score(y_test, LR_pred, average='macro')

print(LR_grid.best_estimator_)
print(test_cm)
print()
print("정확도는 {}%".format(round(test_acc*100, 2)))
print("정밀도는 {}%".format(round(test_prc*100, 2)))
print("재현율은 {}%".format(round(test_rcll*100, 2)))
print("F1는 {}%".format(round(test_f1*100, 2)))

LogisticRegression(C=1000)
[[511 160  13   0]
 [149 286 193  23]
 [ 51 146 334 122]
 [ 10  31 103 547]]

정확도는 62.64%
정밀도는 61.94%
재현율은 62.24%
F1는 62.07%
